In [1]:
import numpy as np
import pandas as pd
import quandl

In [184]:
stock = pd.read_csv('stocks.csv')
stock = stock.iloc[:,1:]

In [2]:
quandl.ApiConfig.api_key = 'mQsh7CqsmzGujFD4cj5y'

In [3]:
test2 = quandl.get_table('WIKI/PRICES', ticker = ['AAPL','MSFT','WMT'],
                        qopts = {'columns':['ticker','date','adj_close']},
                        date = {'gte':'2015-01-01','lte':'2018-12-31'},
                        paginate = True)

In [149]:
clean_data = test2.pivot_table(index = 'date',
                                       columns = 'ticker',
                                       values = 'adj_close')

In [171]:
def GetData(stockList, start, end):
    '''
    Function takes a list of stock codes, the start
    and the end of analysis period as inputs and outputs
    a data frame ready for analysis
    '''
    raw_data = quandl.get_table('WIKI/PRICES', ticker = stockList,
                        qopts = {'columns':['ticker','date','adj_close']},
                        date = {'gte': start,'lte': end},
                        paginate = True)
    
    clean_data = raw_data.pivot_table(index = 'date',
                                       columns = 'ticker',
                                       values = 'adj_close')
    
    # I found sometimes there are NA values in dataframe (eg.AAPL), 
    # so I fill them with column mean
    
    if clean_data.isna().any().any():
        print('NA values in the dataframe')
        clean_data.fillna(clean_data.mean(), inplace = True)
        return clean_data
    
    else:
        return clean_data

In [172]:
test_data = GetData(['AAPL','MSFT','WMT'], '2015-01-01', '2018-12-31')

NA values in the dataframe


In [7]:
def MeanVarCalculation(df):
    '''
    Function takes a dataframe of stock closing prices
    as input and outputs an array of stock mean return 
    and covariance matrix
    '''
    n = df.shape[0]   #n: number of rows
    m = df.shape[1]   #m: number of columns
    array1 = np.array(df.iloc[1:,:])
    array2 = np.array(df.iloc[0:n-1,:])
    
    ret = array1/array2 - 1
    retMean = np.nanmean(ret, axis = 0)
    covMatrix = np.cov(ret, rowvar = False)
    
    return retMean, covMatrix

In [177]:
retMean, covMatrix = MeanVarCalculation(test_data)

In [181]:
covMatrix

array([[3.45420812e-04, 1.15542267e-04, 4.16417563e-05],
       [1.15542267e-04, 2.15862916e-04, 4.65781848e-05],
       [4.16417563e-05, 4.65781848e-05, 1.65815897e-04]])

In [8]:
def WeightCalculation(covMatrix):
    cov_inv = np.linalg.inv(covMatrix)
    b = np.ones(len(covMatrix))
    weight = np.dot(cov_inv , b)
    weight = weight/weight.sum()
    
    return weight

In [183]:
WeightCalculation(covMatrix)

array([0.14851352, 0.31385453, 0.53763195])

In [186]:
r, cov = MeanVarCalculation(stock)

In [187]:
WeightCalculation(cov)

array([ 5.08805994e-02,  3.98667868e-02,  1.47420624e-01,  7.91622513e-02,
        1.60866178e-01,  8.24733144e-02,  1.29257072e-01, -1.01006992e-04,
       -6.06161033e-03,  3.16235792e-01])

In [3]:
sp500 = pd.read_csv('prices.csv')

In [4]:
sp500.head()

MMM    ACE    ABT    ANF   ADBE   AMD   AES    AET    AFL      A  ...    \
0  126.75  30.51  40.05  21.75  25.60  7.01  3.24  42.04  31.59  19.14  ...     
1  126.27  30.69  40.44  21.34  26.30  6.94  3.45  42.33  31.60  19.05  ...     
2  127.19  31.97  40.70  22.11  27.51  7.16  3.45  42.75  32.00  19.96  ...     
3  127.07  31.78  38.87  22.07  28.51  7.17  3.37  42.18  31.73  19.78  ...     
4  125.70  31.17  39.85  22.40  27.13  6.69  3.35  42.23  31.80  18.82  ...     

     WEC   WYNN    XEL   XRX   XLNX     XL   YHOO    YUM    ZMH   ZION  
0  25.65  13.50  11.35  8.16  22.03  80.23  17.60  25.12  40.30  40.64  
1  25.59  13.40  11.54  8.28  22.94  80.70  18.10  24.55  40.40  40.27  
2  26.48  13.29  12.25  8.44  24.24  82.97  18.94  24.99  40.66  41.39  
3  26.05  13.31  11.80  8.31  25.45  82.51  19.15  25.15  40.26  41.59  
4  26.11  13.02  12.60  8.27  24.71  81.47  18.75  25.27  40.00  42.03  

[5 rows x 452 columns]

In [5]:
sp500.shape

(1258, 452)

In [9]:
ret, cov = MeanVarCalculation(sp500)

In [14]:
weight = WeightCalculation(cov)

In [22]:
weight

array([ 1.18281840e-02, -3.03377961e-02, -5.49641209e-03,  2.08445282e-03,
       -5.12574318e-03,  7.30423818e-03, -4.15434584e-03,  2.00000710e-03,
       -1.27079489e-02, -3.86724976e-03, -3.13767922e-03, -1.23088187e-02,
        4.29704276e-03, -6.45858479e-03,  3.78572314e-03, -2.62571423e-02,
        1.04962020e-02,  7.87136105e-02,  8.28488493e-03,  2.21408526e-02,
       -1.12914071e-02,  5.68196758e-02, -3.39074334e-02, -3.87265985e-02,
       -1.60786449e-03,  2.24726299e-02,  2.68281822e-03,  5.34182978e-03,
       -2.44113911e-03, -7.66396887e-04,  1.43982331e-02,  1.77164095e-03,
       -1.64620221e-02,  2.83909699e-02, -2.19913194e-02,  7.57195303e-03,
        2.50756575e-02,  4.00125536e-03,  1.63450790e-02,  9.67853929e-04,
       -6.39340379e-04,  4.95641707e-02,  2.12456042e-02,  3.27052041e-02,
        1.63505405e-03, -2.02177472e-02, -4.66619353e-03,  4.63227959e-03,
        1.14836688e-02, -2.77273713e-03,  2.18172739e-02,  1.88063874e-02,
       -3.82049659e-02, -

In [30]:
indexes = []
for index,value in enumerate(weight):
    if value < 0:
        indexes.append(names[index])

In [26]:
sp500.columns

Index(['MMM', 'ACE', 'ABT', 'ANF', 'ADBE', 'AMD', 'AES', 'AET', 'AFL', 'A',
       ...
       'WEC', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XL', 'YHOO', 'YUM', 'ZMH',
       'ZION'],
      dtype='object', length=452)

In [27]:
names = list(sp500.columns)

In [32]:
len(indexes)

214